## WinGO - A Wingsail Parameterisation and Optimisation Model

Please make sure to run the notebook with Python 3.11 in a Windows or Linux  (Debian 12 tested) environment, and pre-install the following libraries with conda or pip before getting start:

`- requests - rich - numpy - scipy - matplotlib - typing - numpy-stl -ipywidgets`

### Parameters 3D visualisation and model generation

In this project, nineteen parameters has been chosen to define the characteristics of the wingsail geometry. The first two parameters define the wingsail overall span and the clearance to the sea level; the rest parameters difined the characteristics of five key cross-sections along the wingsail, including their location information, chord, airfoil profile and offest over the base.

In the following block, three sets of wingsail parameters samples are prepared for testing. The cross-sections can be visualised with matplotlib and the wingsail parameters can be generated to an .stl format 3D model.

Beyond a better visibility by inspecting the generated 3D model based on given datapoints, this module is also essential in the future steps by have an anti-generation geometry with the conditional diffusion model optimisation.

>***Change the constants to get start with a wingsail 3D model generation and lift and drag ratio calculation.***

In [ ]:
from Wingsail_AllinOne import Geometrisation as WinGeo
from datetime import datetime
import os

# Wingsail sample data, will be expanded to a dataset of 5000 datapoints in the future.
sample_wingsail_0 = [4, 0, 1, 15, 0.2, 1, 15, 0, 0.5, 1, 15, 0, 0.8, 1, 15, 0, 1, 15, 0]
oracle_ac72 = [40, 3, 9, 20, 0.26, 1.03, 17, -0.15, 0.52, 0.96, 15, -0.21, 0.78, 0.76, 15, -0.15, 0.38, 12, 0.06]
sample_wingsail_2 = [3.5, 1, 1, 21, 0.5, 1.05, 21, 0, 0.8, 0.8, 18, 0.05, 0.93, 0.6, 15, 0.2, 0.35, 10, 0.4]
sample_wingsail_3 = [60, 20, 15, 22, 0.45, 0.95, 20, 0.02, 0.75, 0.8, 20, 0.05, 0.95, 0.66, 17, 0.09, 0.4, 12, 0.16]

# Density level of cloud points. Higher refinement level means a higher resolution of the wingsail model.
INPUT_SAMPLE = oracle_ac72
REFINEMENT_LEVEL = 1
UNIFORM_BOOL = True

# Instantiating the Wingsail Geometrisation class
wingsail = WinGeo(INPUT_SAMPLE, REFINEMENT_LEVEL, UNIFORM_BOOL)

# Plot the wingsail geometry with matplotlib
# plot = wingsail.plot3d()

# Generate the 3D model
if wingsail.uniform_bool:
    output_uniform_str = "_Uniformed_"
else:
    output_uniform_str = "_Customised_"
output_filename = "Wingsail" + output_uniform_str + datetime.now().strftime("%y%m%d%H%M%S") + "_RL" + str(REFINEMENT_LEVEL) + ".stl"
save_folder = os.path.join(os.getcwd(), "stl_output")
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
save_path = os.path.join(save_folder, output_filename)
print(f"Saving 3D model to {save_path}")
wingsail.generate_stl(save_path)

### Thrust Calculation for dataset preparation (Xfoil method):

This module calculates the maximum trust provided by a wingsail geometry based under given wind conditions. The $C_l$ and $C_d$ values for each 2D sections are calculated by [Xfoil](https://web.mit.edu/drela/Public/web/xfoil/) software.

In [ ]:
from Wingsail_AllinOne import Calculation as WinCal
from rich import print
import numpy as np
import time
t = time.perf_counter()

# Export a sample wingsail geometry
sample_wingsail_0 = [4, 0, 1, 15, 0.2, 1, 15, 0, 0.5, 1, 15, 0, 0.8, 1, 15, 0, 1, 15, 0]
oracle_ac72 = [40, 3, 9, 20, 0.26, 1.03, 17, -0.15, 0.52, 0.96, 15, -0.21, 0.78, 0.76, 15, -0.15, 0.38, 12, 0.06]
sample_wingsail_2 = [3.5, 1, 1, 21, 0.5, 1.05, 21, 0, 0.8, 0.8, 18, 0.05, 0.93, 0.6, 15, 0.2, 0.35, 10, 0.4]
sample_wingsail_3 = [60, 20, 15, 22, 0.45, 0.95, 20, 0.02, 0.75, 0.8, 20, 0.05, 0.95, 0.66, 17, 0.09, 0.4, 12, 0.16]

SAILING_CONDITION = [7, 8, 50]
AOA_RANGE = [1, 25, 1]
XFOIL_ITER = 200
REFINEMENT_LEVEL = 1

wincal = WinCal(oracle_ac72, [], REFINEMENT_LEVEL) # For Calculation class test
max_thrust, max_angle_of_attack = wincal.max_thrust_with_xfoil(SAILING_CONDITION, AOA_RANGE, XFOIL_ITER)
# max_thrust, max_angle_of_attack = wincal.debug(SAILING_CONDITION)

print(f"\n:sailboat: Vessel speed: {SAILING_CONDITION[0]}")
print(f":triangular_flag_on_post: Wind condition: {SAILING_CONDITION[1]} m/s at {SAILING_CONDITION[2]} deg")
print(f":smiley: Max thrust of {np.round(max_thrust, 3)} N found at angle of attack: {max_angle_of_attack} deg\n")

print(f":clock1: Cost time: {time.perf_counter() - t:8f} seconds!\n")

### Thrust Calculation for dataset preparation (NeuralFoil method):

This module calculates the maximum trust provided by a wingsail geometry based under given wind conditions. The $C_l$ and $C_d$ values for each 2D sections are calculated by [Neuralfoil](https://github.com/peterdsharpe/NeuralFoil) model, with the model size of "xxxlarge", the calculation time consumption is 10 times faster than Xfoil with an error rate of 0.1%. (arround 2.2s per sea condition condition with 25 AOA inputs.)


In [ ]:
from Wingsail_AllinOne import Calculation as WinCal
import numpy as np
import time

t = time.perf_counter()

# Export a sample wingsail geometry
sample_wingsail_0 = [4, 0, 1, 15, 0.2, 1, 15, 0, 0.5, 1, 15, 0, 0.8, 1, 15, 0, 1, 15, 0]
sample_wingsail_1 = [4, 0, 1, 15, 0.2, 1, 15, 0, 0.5, 1, 15, 0, 0.8, 1, 12, 0, 0.5, 10, 0]
sample_wingsail_2 = [3.5, 1, 1, 21, 0.5, 1.05, 21, 0, 0.8, 0.8, 18, 0.05, 0.93, 0.6, 15, 0.2, 0.35, 10, 0.4]
sample_wingsail_3 = [60, 30, 15, 22, 0.45, 0.95, 20, 0.02, 0.75, 0.8, 20, 0.05, 0.95, 0.66, 17, 0.09, 0.4, 12, 0.16]
oracle_ac72 = [40, 3, 9, 20, 0.26, 1.03, 17, -0.15, 0.52, 0.96, 15, -0.21, 0.78, 0.76, 15, -0.15, 0.38, 12, 0.06]

# Define physical constants
VESSEL_SPEED_RANGE = [5, 10, 1]
TRUE_WIND_SPEED_RANGE = [6, 9, 1]
TRUE_WIND_ANGLE_RANGE = [30, 150, 10]
AOA_RANGE = [1, 25, 1]
WINGSAIL_MODEL = oracle_ac72
REFINEMENT_LEVEL = 1
NEURALFOIL_NETWORL = "large"

with open("neuralfoil_data.txt", "w") as f:
    f.write(f"Wingsail parameter: {str(WINGSAIL_MODEL)}\n")
    f.write("===========================================================================================================\n")
    
input_vs_range = [VESSEL_SPEED_RANGE[0], VESSEL_SPEED_RANGE[1] + VESSEL_SPEED_RANGE[2], VESSEL_SPEED_RANGE[2]]
input_tws_range = [TRUE_WIND_SPEED_RANGE[0], TRUE_WIND_SPEED_RANGE[1] + TRUE_WIND_SPEED_RANGE[2], TRUE_WIND_SPEED_RANGE[2]]
input_twa_range = [TRUE_WIND_ANGLE_RANGE[0], TRUE_WIND_ANGLE_RANGE[1] + TRUE_WIND_ANGLE_RANGE[2], TRUE_WIND_ANGLE_RANGE[2]]

wincal = WinCal(WINGSAIL_MODEL, [], REFINEMENT_LEVEL)

for vessel_speed in range(*input_vs_range):
    for true_wind_speed in range(*input_tws_range):
        for true_wind_angle in range(*input_twa_range):
            SAILING_CONDITION = [vessel_speed, true_wind_speed, true_wind_angle]
            max_thrust, max_angle_of_attack = wincal.max_thrust_with_neuralfoil(SAILING_CONDITION, AOA_RANGE, NEURALFOIL_NETWORL)
            with open("neuralfoil_data.txt", "a") as f:
                f.write(f"Vessel speed {vessel_speed} m/s,")
                f.write(f"wind {true_wind_speed} m/s from {true_wind_angle} deg: ")
                f.write(f"Max thrust of {np.round(max_thrust, 3)} N found at aoa {max_angle_of_attack} deg\n")

print(f":clock1: Cost time: {time.perf_counter() - t:8f} seconds!\n")
